In [1]:
# Colab settings
# !wget --header="Host: archive.ics.uci.edu" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7" --header="Referer: https://archive.ics.uci.edu/ml/machine-learning-databases/00363/" "https://archive.ics.uci.edu/ml/machine-learning-databases/00363/Dataset.zip" -O "Dataset.zip" -c
# !unzip Dataset.zip
# !git clone https://github.com/janchk/ML_HOMEWORK1
# from ML_HOMEWORK_1.models import LinearRegressionWithGd
# from ML_HOMEWORK_1.common import *

In [2]:
#import part 
import matplotlib.pyplot as plt
from IPython import display
from models import LinearRegressionWithGd
from common import *
import pandas as pd
path = "Dataset/Training/"

In [3]:
# configuration
num_folds = 5
epochs = 1000
learning_rate = 0.1

lrg = LinearRegressionWithGd()
lrg.learning_rate = learning_rate

df_1, df_2 = folding(path)
np.random.shuffle(df_1)


In [4]:
def make_table(stats, loss, betas, bias):
    columns = ["Val MSE", "Val R2", "Val RMSE", "Val R2/RMSE"] + [f"p{p+1}" for p in range(len(betas))] + ["bias"]
    # mean = np.mean([np.sqrt(loss)] + [stats] + [betas] + [bias])
    # mean = np.mean(np.sqrt(loss) + stats + betas + bias)
    # std = np.std(np.sqrt(loss) + stats + betas + bias)
    index = [f"Fold {i+1}" for i, _ in enumerate(stats) ]
    df = pd.DataFrame(np.array([loss] + [stats] + [betas] + [bias]),
                  columns=columns,
                  index=index)
    display.display(df)   
    
    pass

### Training

In [5]:
features = []
bias = []
stats = []
losses = [] #loss per fold
for i in range(num_folds):
    train, test = cv_loo(df_1, num_folds, i)

    X_train = normalized(train.T[0:53]).T
    X_test = normalized(test.T[0:53]).T

    Y_train = train.T[53].T
    Y_train = np.expand_dims(Y_train, axis=-1)

    Y_test = test.T[53].T
    Y_test = np.expand_dims(Y_test, axis=-1)

    loss = lrg.train(epochs, X_train, Y_train)
    stat = lrg.validate(X_test, Y_test)
    losses.append(loss)
    stats.append(stat)
    features.append(lrg.betas)
    bias.append(*lrg.bias)
    
    # graphs_plot(losses, stats[0], stats[1], stats[2])

mean_stats = np.sum(stats, axis=0) / num_folds
print("mse_val is {}, r2_val is {}, rmse is {}, r2/rmse {}".format(
    mean_stats[0], mean_stats[1], mean_stats[2], mean_stats[3]))


Loss: 1011.9705: 100%|██████████| 1000/1000 [01:33<00:00, 10.73it/s]


val loss is 1270.1612508211983, R2 is 0.14132154210757586, RMSE is 35.63932169417929, R2/RMSE is 0.003965326369571643
val loss is 1167.6259674663008, R2 is 0.1787383922558362, RMSE is 34.17054239350468, R2/RMSE is 0.005230774220599195
val loss is 936.4149529333158, R2 is 0.1254947497481238, RMSE is 30.600897910573078, R2/RMSE is 0.0041010152746126915
val loss is 921.8500468447003, R2 is 0.1637367854227535, RMSE is 30.361983578888587, R2/RMSE is 0.005392822408895695
val loss is 1183.9987252333904, R2 is 0.16816152774190074, RMSE is 34.40928254458948, R2/RMSE is 0.004887097762761766
mse_val is 1096.010188659781, r2_val is 0.15549059945523802, rmse is 33.03640562434702, r2/rmse 0.004715407207288198


### Features



In [62]:
columns = ["bias"] + [f"f{f+1}" for f in range(len(features[0]))]
index = [f"Fold {i+1}" for i, _ in enumerate(features)] + ["mean"] + ["std"]

bias_np = np.expand_dims(np.squeeze(bias), 1)
features_np = np.squeeze(features, axis=2)
features_np = np.hstack((bias, features_np))
features_mean = np.mean(features_np, axis=0)
features_std = np.std(features_np, axis=0)
df = pd.DataFrame(np.vstack((features_np, features_mean, features_std )),
                  columns=columns,
                  index=index)

display.display(df)    



,bias,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53
Fold 1,4.362976,15.501874,7.906394,40.502210,-9.080475,40.548921,38.269390,69.393365,75.009377,54.028792,...,-0.084542,-0.892554,-1.764078,-1.814654,-0.058019,2.262402,5.404611,0.080201,-0.622918,-1.986087
Fold 2,2.630691,23.555008,11.517066,66.553398,-14.973420,66.989997,61.929655,118.242327,127.810572,90.081750,...,-1.434715,-1.991464,-2.704612,-3.549457,-0.195917,3.314455,8.233231,0.136921,-2.457493,-2.250175
Fold 3,1.405310,29.860006,11.735421,84.746847,-18.058537,88.936926,76.366006,155.695802,170.609669,115.783138,...,-3.164173,-2.194861,-3.867728,-5.467131,0.268366,4.821364,11.119688,-1.449363,-2.988197,-3.012123
Fold 4,0.534440,32.850267,9.138710,94.286941,-18.911005,105.414033,82.776994,180.041109,199.882572,130.221919,...,-3.871227,-2.392304,-4.609277,-7.389027,0.619703,6.867882,13.440071,-2.618452,-4.628768,-2.941750
Fold 5,0.238595,33.147260,6.390099,98.431807,-18.527476,110.424638,84.094901,193.411372,216.533000,137.478019,...,-4.118141,-2.657648,-5.424940,-8.181680,1.359137,7.309542,15.680732,-3.950255,-5.086451,-3.721195
mean,1.834402,26.982883,9.337538,76.904241,-15.910183,82.462903,68.687389,143.356795,157.969038,105.518724,...,-2.534560,-2.025766,-3.674127,-5.280390,0.398654,4.915129,10.775667,-1.560189,-3.156766,-2.782266
std,1.513168,6.697275,2.062779,21.257228,3.687628,25.871678,17.121864,44.955036,51.260328,30.418796,...,1.542924,0.607876,1.308860,2.362463,0.556854,1.957262,3.649364,1.575797,1.601765,0.612679


### Stats



In [56]:
stats_np = np.array(stats)
losses_np = np.expand_dims(np.array(losses), 1)
columns = ["mse_train"] + ["mse_val"] + ["r2_val"] + ["rmse_val"] + ["r2/rmse_val"]
index = [f"Fold {i+1}" for i, _ in enumerate(stats_np)] # + ["mean"] + ["std"]
all_stats = np.hstack((losses_np, stats_np))
df = pd.DataFrame(all_stats,
                  columns=columns,
                  index=index)

display.display(df) 

,mse_train,mse_val,r2_val,rmse_val,r2/rmse_val
Fold 1,1192.737775,1270.161251,0.141322,35.639322,0.003965
Fold 2,1122.014951,1167.625967,0.178738,34.170542,0.005231
Fold 3,1146.093977,936.414953,0.125495,30.600898,0.004101
Fold 4,1114.192491,921.850047,0.163737,30.361984,0.005393
Fold 5,1020.636702,1183.998725,0.168162,34.409283,0.004887


In [59]:
print(np.squeeze(bias))

[4.3629756  2.6306914  1.40531004 0.53443952 0.23859465]


MD
